# Computational Genomics - Project 2
# Comparison of our results with Arrowhead - calculating metrics
## Authors: Kacper Grzymkowski, Mikołaj Malec, Piotr Marciniak
# Loading libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
import os

In [13]:
def read_juicer_bedpe(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, sep="\t", skiprows=[1])
    df.rename(columns={"#chr1": "chr1"}, inplace=True)
    return df

def read_bedpe(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, sep="\t")
    return df


arrowhead_results = {}
for dir_name in os.listdir("results"):
    if os.path.isdir(os.path.join("results", dir_name)):
        arrowhead_results[dir_name] = {}
        for file in os.listdir(os.path.join("results", dir_name)):
            if file.endswith(".bedpe"):
                res = read_juicer_bedpe(os.path.join("results", dir_name, file))
                res["geometry"] =[Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]) for x1, y1, x2, y2 in zip(res["x1"], res["y1"], res["x2"], res["y2"])]
                arrowhead_results[dir_name][file[:-6]] = gpd.GeoDataFrame(res)
arrowhead_results

{'inter_30': {'10000_blocks':     chr1         x1         x2 chr2         y1         y2 name score strand1  \
  0     10   27570000   28390000   10   27570000   28390000    .     .       .   
  1     10   15870000   16810000   10   15870000   16810000    .     .       .   
  2     10   30650000   31820000   10   30650000   31820000    .     .       .   
  3     10    3200000    4730000   10    3200000    4730000    .     .       .   
  4     10   74240000   75090000   10   74240000   75090000    .     .       .   
  ..   ...        ...        ...  ...        ...        ...  ...   ...     ...   
  212    9  111660000  112900000    9  111660000  112900000    .     .       .   
  213    9  120600000  121400000    9  120600000  121400000    .     .       .   
  214    9   19930000   21140000    9   19930000   21140000    .     .       .   
  215    9   99150000   99830000    9   99150000   99830000    .     .       .   
  216    9   37910000   38070000    9   37910000   38070000    .     .

In [12]:
our_results = {}

for file in os.listdir("output"):
    res = read_bedpe(os.path.join("output", file))
    res["geometry"] = [Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]) for x1, y1, x2, y2 in zip(res["x1"], res["y1"], res["x2"], res["y2"])]
    key = file.split("_")[0]
    key = key if key != "inter" else "inter_30"
    our_results[key] = gpd.GeoDataFrame(res)
    
our_results

{'inter_30':     chr1        x1        x2 chr2        y1        y2  \
 0     19  19320000  24415000   19  19320000  24415000   
 1     19  27245000  33685000   19  27245000  33685000   
 2     19  32860000  35740000   19  32860000  35740000   
 3     19  34310000  39375000   19  34310000  39375000   
 4     19  42290000  46325000   19  42290000  46325000   
 ..   ...       ...       ...  ...       ...       ...   
 236    8  44100000  57540000    8  44100000  57540000   
 237    8  56920000  68035000    8  56920000  68035000   
 238    8  67435000  75585000    8  67435000  75585000   
 239    8  74355000  86605000    8  74355000  86605000   
 240    8  86150000  94000000    8  86150000  94000000   
 
                                               geometry  
 0    POLYGON ((19320000.000 19320000.000, 24415000....  
 1    POLYGON ((27245000.000 27245000.000, 33685000....  
 2    POLYGON ((32860000.000 32860000.000, 35740000....  
 3    POLYGON ((34310000.000 34310000.000, 39375000....  


In [22]:
import warnings
warnings.filterwarnings("ignore")

def intersection_over_union(first: gpd.GeoDataFrame, our_gdf: gpd.GeoDataFrame) -> float:
    # Calculate intersection area
    intersection_area = gpd.overlay(first, our_gdf, how='intersection').geometry.area.sum()

    # Calculate union area
    union_area = gpd.overlay(first, our_gdf, how='union').geometry.area.sum()

    return intersection_area / union_area


def intersections(first: gpd.GeoDataFrame, second: gpd.GeoDataFrame) -> int:
    # Calculate intersection 
    intersections = gpd.overlay(first, second, how='intersection').shape[0]

    return intersections

comparison = {
    "dataset": [],
    "resolution": [],
    "iou": [],
    "intersections": []
}

for key, value in arrowhead_results.items():
    if key not in our_results:
        continue
    first = our_results[key]
    for key2, second in value.items():
        comparison["dataset"].append(key)
        comparison["resolution"].append(key2)
        comparison["iou"].append(intersection_over_union(first, second))
        comparison["intersections"].append(intersections(first, second))

comparison_df = pd.DataFrame(comparison)
comparison_df

,dataset,resolution,iou,intersections
0,inter_30,10000_blocks,0.052127,3395
1,inter_30,5000_blocks,0.000299,88
2,inter_30,25000_blocks,0.394424,28436
3,4DNFI1UEG1HD,10000_blocks,0.203257,2243
4,4DNFI1UEG1HD,5000_blocks,0.191931,2610
5,4DNFI1UEG1HD,25000_blocks,0.224319,1236
6,ENCFF629KXF,10000_blocks,0.003028,41
7,ENCFF629KXF,25000_blocks,0.068429,192
